# Model MLP-CNN untuk Rekonstruksi Citra pada Sistem Continuous Wave Diffuse Optical Tomography (CW-DOT)

## Mengimpor semua library yang dibutuhkan

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import requests
import seaborn as sns
import tensorflow as tf
from io import BytesIO
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.image import ssim
from tensorflow.keras.applications import VGG19
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from sklearn.model_selection import train_test_split

## Memuat dan mempses semua dataset yang akan digunakan

In [ ]:
folder1_path = "-"
folder2_path = "-"
foldergmb2_path = "-"

# Load file Excel (anomali), Kode untuk memuat data bisa disesuaikan untuk lokal atau bukan
G1_list = []
# epsilon = 1e-6
for file_name in os.listdir(folder1_path):
    if file_name.endswith(".xlsx"):
        file_path = os.path.join(folder1_path, file_name)
        data = pd.read_excel(file_path, header=None).to_numpy().flatten()
        data_scaled = (data - np.mean(data)) / np.mean(data)
        data_norm = data_scaled/np.max(data_scaled)
        G1_list.append(data_norm.reshape(-1, 1))
G1 = np.array(G1_list).squeeze()

# --- Muat Gambar Objek Asli (Persiapan Data) ---
image2_list = []
try:
    if not os.path.exists(folder2_path) or not os.listdir(folder2_path):
        raise FileNotFoundError("Folder tidak ditemukan atau kosong.")

    for filename in sorted(os.listdir(folder2_path)):
        if filename.lower().endswith((".png", ".jpg", ".jpeg")):
            img_path = os.path.join(folder2_path, filename)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Pastikan RGB
                img = cv2.resize(img, (100, 100))        # Sesuaikan ukuran (sesuai kebutuhan)
                img = img / 255.0                        # Normalisasi ke 0-1
                image2_list.append(img)
    if not image2_list:
        raise ValueError("Tidak ada gambar yang valid ditemukan di folder.")

    image2_array = np.array(image2_list)
    print(f"Dimuat {len(image2_list)} gambar dari '{folder2_path}'. Bentuk: {image2_array.shape}")

except (FileNotFoundError, ValueError) as e:
    print(f"Error: {e}. Membuat dummy image2_array untuk demonstrasi.")
    # Fallback: Buat dummy data jika tidak ada gambar
    num_dummy_images = 3
    image2_array = np.random.rand(num_dummy_images, 100, 100, 3).astype(np.float32)
    print(f"Dummy image2_array dibuat dengan bentuk: {image2_array.shape}")

# --- Proses Konversi Gambar ke Hitam Putih (Biner) ---
binary_images_list = []
threshold_value = 0.1 # Nilai threshold (antara 0.0 - 1.0). Sesuaikan jika perlu.

for img_rgb_normalized in image2_array:
    # 1. Konversi ke Skala Abu-abu (menggunakan bobot umum)
    grayscale_img = (0.2989 * img_rgb_normalized[:, :, 0] +
                     0.5870 * img_rgb_normalized[:, :, 1] +
                     0.1140 * img_rgb_normalized[:, :, 2])

    # 2. Terapkan Thresholding untuk membuat gambar biner (hitam/putih)
    # Piksel > threshold_value menjadi 1 (putih), lainnya 0 (hitam)
    binary_img = (grayscale_img > threshold_value).astype(np.float32)
    binary_images_list.append(binary_img)

binary_images_array = np.array(binary_images_list)

print(f"Bentuk array gambar biner yang dihasilkan: {binary_images_array.shape}")

# --- Tampilkan Hasil (Hanya Beberapa Contoh) ---
# Tampilkan maksimal 3 pasang gambar untuk ilustrasi
# num_display = min(3, len(image2_array))

# plt.figure(figsize=(num_display * 5, 8))

# for i in range(num_display):
#     # Gambar Asli (Berwarna)
#     plt.subplot(2, num_display, i + 1)
#     plt.imshow(image2_array[i])
#     plt.title(f'Asli {i+1}')
#     plt.axis('off')

#     # Gambar Hitam Putih (Biner)
#     plt.subplot(2, num_display, i + 1 + num_display)
#     plt.imshow(binary_images_array[i], cmap='gray') # Penting: gunakan cmap='gray'
#     plt.title(f'Biner {i+1} (Th: {threshold_value:.2f})')
#     plt.axis('off')

# plt.tight_layout()
# plt.show()

# Load gambar (Linear Satu Langkah)
image3_list = []
for filename in sorted(os.listdir(foldergmb2_path)):
    if filename.endswith((".png", ".jpg", ".jpeg")):
        img_path = os.path.join(foldergmb2_path, filename)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (100, 100)) # aslinya adalah (100, 100)
        img = img / 255.0
        if img is not None:
            image3_list.append(img)
image3_array = np.array(image3_list)

Membagi semua dataset menjadi data latih dan data uji dengan perbandingan 8:2

In [ ]:
### Persiapan Data Latih dan Data Uji untuk model
X_train1, X_test1, Y_train1, Y_test1 = train_test_split(G1, binary_images_array, test_size=0.2, random_state=42)

# ### Persiapan untuk membandingkan dengan data rekonstruksi Linear Satu Langkah
# X_train2, X_test2, Y_train2, Y_test2 = train_test_split(image_array, image2_array, test_size=0.2, random_state=42)

X_train1 = np.expand_dims(X_train1, axis=-1)
X_test1 = np.expand_dims(X_test1, axis=-1)
Y_train1 = np.expand_dims(Y_train1, axis=-1)
Y_test1 = np.expand_dims(Y_test1, axis=-1)
# print(X_train1.shape)
# print(X_test1.shape)
# print(Y_train1.shape)
# print(Y_test1.shape)

## Arsitektur, Pelatihan, dan Optimasi Model yang digunakan

Model yang digunakan adalah kombinasi dari MLP (Multi-Layer Perceptn) dan CNN (Convolutional Neural Network)

MLP --> digunakan untuk mempses/mengonversi data distribusi intensitas cahaya menjadi sebuah sketsa gambar awal berdasarkan data label/data gambar objek asli yang digunakan

CNN --> digunakan untuk merekonstruksi dan mempercantik gambar yang dihasilkan berdasarkan keluaran dari MLP

In [ ]:
# Dimensi input dan output
num_input_features = 256  # Input
image_shape = (100, 100,1)  # Target gambar

# 1. MLP, layer dan neuron bisa diatur sesuai dengan kebutuhan
def build_mlp(input_dim, latent_dim):
    inputs = layers.Input(shape=(input_dim,))
    x = layers.Dense(2048, activation='relu')(inputs)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(4096, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(latent_dim[0] * latent_dim[1] * latent_dim[2], activation='relu')(x)
    outputs = layers.Reshape(latent_dim)(x)
    return models.Model(inputs, outputs)

# 2. CNN, layer dan neuron bisa diatur sesuai dengan kebutuhan
def build_cnn(latent_dim, output_shape):
    inputs = layers.Input(shape=latent_dim)
    x = layers.Conv2DTranspose(128, (3, 3), strides=2, padding='same', activation='relu')(inputs)
    x = layers.Conv2DTranspose(64, (3, 3), strides=2, padding='same', activation='relu')(x)
    x = layers.Conv2DTranspose(32, (3, 3), strides=2, padding='same', activation='relu')(x)
    x = layers.Cropping2D(((2,2), (2,2)))(x)
    outputs = layers.Conv2DTranspose(1, (3, 3), strides=1, padding='same', activation='sigmoid')(x)
    return models.Model(inputs, outputs)

# 3. Bangun Model MLP + CNN
latent_dim = (13, 13, 128)  # Bentuk cetak biru yang akan dihasilkan oleh MLP
mlp_encoder = build_mlp(num_input_features, latent_dim)
cnn_decoder = build_cnn(latent_dim, image_shape)

# 4. Hubungkan MLP dan CNN
inputs = layers.Input(shape=(num_input_features,))
latent_output = mlp_encoder(inputs)
outputs = cnn_decoder(latent_output)

vgg = VGG19(weights="imagenet", include_top=False)
vgg.trainable = False
loss_model = Model(inputs=vgg.input, outputs=vgg.get_layer("block3_conv3").output)

def perceptual_loss(y_true, y_pred):
    y_true_rgb = tf.image.grayscale_to_rgb(y_true)
    y_pred_rgb = tf.image.grayscale_to_rgb(y_pred)
    return tf.reduce_mean(tf.abs(loss_model(y_true_rgb) - loss_model(y_pred_rgb)))
def ssim_loss(y_true, y_pred):
    return 1 - tf.image.ssim(y_true, y_pred, max_val=1.0)
def combined_loss(y_true, y_pred):
    return 0.4 * tf.keras.losses.MeanSquaredError()(y_true, y_pred) + 0.3 * ssim_loss(y_true, y_pred) +  0.3 * perceptual_loss(y_true, y_pred)

# **4. Compile Model**
model = models.Model(inputs, outputs)
model.compile(optimizer='adam', loss=combined_loss, metrics=['mse'])  # Bisa pakai loss lain seperti SSIM jika ingin kualitas gambar lebih baik

# **5. Cek Arsitektur Model**
model.summary()

In [ ]:
# Callback untuk mengurangi learning rate saat val_loss stagnan
lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=5, verbose=1
)

# Callback untuk menghentikan training jika tidak ada perbaikan
early_stop = EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True, verbose=1
)

# Training model, parameter di bawah ini bisa diatur sesuai kebutuhan
history = model.fit(
    X_train1, Y_train1,
    epochs=75,
    batch_size=32,
    validation_split=0.2,
    callbacks=[lr_scheduler, early_stop]
)

## Hasil dari model yang digunakan

In [ ]:
# Prediksi gambar, ganti cmap=gray jika ingin gambar grayscale dan ganti cmap=jet jika ingin gambar berwarna
num_samples = 15
X_test_sample = X_test1[:num_samples]  # Contoh data input
Y_test_sample = Y_test1[:num_samples]
Y_pred_sample = model.predict(X_test1)

# print(Y_test_sample.shape)
# print(Y_pred_sample.shape)

# Tampilkan hasilnya
fig, axes = plt.subplots(num_samples, 2, figsize=(10, 2 * num_samples))

for i in range(num_samples):
    true_img = Y_test_sample[i]
    pred_img = Y_pred_sample[i]

    # Gambar asli (target)
    axes[i, 0].imshow(true_img, cmap='gray') # Ganti dengan cmap='jet' jika ingin berwarna
    axes[i, 0].set_title("Target (Asli)")
    axes[i, 0].axis("off")
    
    # Gambar hasil prediksi
    axes[i, 1].imshow(pred_img, cmap='gray')
    axes[i, 1].set_title("Prediksi (Model)")
    axes[i, 1].axis("off")

plt.tight_layout()
plt.show()

Grafik penurunan loss dan MSE

In [ ]:
# Plot Loss
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss', color='blue')
plt.plot(history.history['val_loss'], label='Validation Loss', color='orange')
plt.title('Loss vs. Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

# Plot MSE
plt.subplot(1, 2, 2)
plt.plot(history.history['mse'], label='Training MSE', color='green')
plt.plot(history.history['val_mse'], label='Validation MSE', color='red')
plt.title('MSE vs. Epoch')
plt.xlabel('Epoch')
plt.ylabel('Mean Squared Err')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

Hasil metrik SSIM dan PSNR beserta grafik distribusinya (lebih baik ganti PSNR dengan Dice Coefficient atau DSC)

In [ ]:
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

ssim_scores = []
psnr_scores = []

for y_true, y_pred in zip(Y_test1, Y_pred_sample):
    ssim_score = ssim(y_true, y_pred, channel_axis=-1, data_range=1.0, win_size=11)
    psnr_score = psnr(y_true, y_pred, data_range=1.0)
    
    ssim_scores.append(ssim_score)
    psnr_scores.append(psnr_score)

print("Average SSIM:", np.mean(ssim_scores))
print("Average PSNR:", np.mean(psnr_scores))

# Plot histogram
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(ssim_scores, bins=20, color='skyblue', edgecolor='black')
plt.title("Distribusi SSIM")
plt.xlabel("SSIM")
plt.ylabel("Frekuensi")

plt.subplot(1, 2, 2)
plt.hist(psnr_scores, bins=20, color='salmon', edgecolor='black')
plt.title("Distribusi PSNR")
plt.xlabel("PSNR (dB)")
plt.ylabel("Frekuensi")

plt.tight_layout()
plt.show()

## Perbandingan hasil antara Linear Satu Langkah dan MLP-CNN

In [ ]:
num_samples     = 15
X_test_sample   = X_test1[:num_samples]
Y_true_sample   = Y_test1[:num_samples]     # gambar asli (target)
Y_lin_sample    = Y_test2[:num_samples]     # rekonstruksi linear 1 langkah
Y_pred_sample   = model.predict(X_test_sample)[:num_samples]   # prediksi MLP‑CNN

# -------------------------------------------------
# Plot 3 kolom  (Asli | Linear 1 Langkah | Prediksi)
# -------------------------------------------------
fig, axes = plt.subplots(num_samples, 3, figsize=(12, 3 * num_samples))

for i in range(num_samples):
    # ---- kolom 0: gambar asli ----
    axes[i, 0].imshow(np.clip(Y_true_sample[i], 0, 1))
    axes[i, 0].set_title("Target (Asli)")
    axes[i, 0].axis("off")

    # ---- kolom 1: linear 1 langkah ----
    axes[i, 1].imshow(np.clip(Y_lin_sample[i], 0, 1))
    axes[i, 1].set_title("Linear 1 Langkah")
    axes[i, 1].axis("off")

    # ---- kolom 2: prediksi MLP-CNN ----
    axes[i, 2].imshow(np.clip(Y_pred_sample[i], 0, 1))
    axes[i, 2].set_title("Prediksi MLP‑CNN")
    axes[i, 2].axis("off")

# Rapikan spasi antarkolom
plt.tight_layout()
plt.show()


## Menyimpan dan Memuat Model

In [ ]:
# Simpan Model
model.save('-')  # Format HDF5 atau Keras

In [ ]:
# Memuat model untuk format Keras
vgg = VGG19(weights="imagenet", include_top=False)
vgg.trainable = False
loss_model = Model(inputs=vgg.input, outputs=vgg.get_layer("block3_conv3").output)

def perceptual_loss(y_true, y_pred):
    return tf.reduce_mean(tf.abs(loss_model(y_true) - loss_model(y_pred)))
def ssim_loss(y_true, y_pred):
    return 1 - tf.image.ssim(y_true, y_pred, max_val=1.0)
def combined_loss(y_true, y_pred):
    return 0.4 * tf.keras.losses.MeanSquaredErr()(y_true, y_pred) + 0.3 * ssim_loss(y_true, y_pred) +  0.3 * perceptual_loss(y_true, y_pred)

# Format Keras
model = load_model("C:/Users/TALITA/SSSKRIPPPSI/mlp_cnn_model100.keras", custom_objects={
    'combined_loss': combined_loss,
    'perceptual_loss': perceptual_loss,
    'ssim_loss': ssim_loss
})

model.summary()

## Percobaan dengan data eksperimen

In [ ]:
folder_eks = "-"

Eks_list = []
epsilon = 1e-6
file_names = []
for file_name in os.listdir(folder_eks):
    if file_name.endswith(".xlsx"):
        file_path = os.path.join(folder_eks, file_name)
        data = pd.read_excel(file_path, header=None).to_numpy().flatten()
        data = data.reshape(-1, 1)  # reshape ke 2D
        data_scaled = (data - np.mean(data))/ np.mean(data)
        data_norm = data_scaled/np.max(data_scaled)
        Eks_list.append(data_scaled)  # flatten lagi supaya array 1D
        file_names.append(file_name)

# Gabungkan semua data
Eks = np.array(Eks_list)
# Jika model butuh input shape tertentu, sesuaikan
print("Eks shape:", Eks.shape)  # (jumlah_sample, fitur)
print(Eks[1])

# Prediksi
C = model.predict(Eks)

In [ ]:
# Tampilkan 3 hasil prediksi
num_display = 3
plt.figure(figsize=(10, 3 * num_display))

for i in range(num_display):
    plt.subplot(num_display, 1, i+1)
    plt.imshow(C[i], cmap='jet')
    plt.axis('off')
    plt.title(f"Hasil Rekonstruksi: {file_names[i]}")

plt.tight_layout()
plt.show()

# Dashboard untuk menampilkan semua hasil dari MLP-CNN dan perbandingannya dengan Linear Satu Langkah, pisah dan jalankan di file tersendiri untuk dashboard

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import streamlit as st
import tensorflow as tf
import threading
import time
fm PIL import Image
fm sklearn.model_selection import train_test_split
fm sklearn.prepcessing import MinMaxScaler
fm skimage.metrics import peak_signal_noise_ratio as psnr
fm skimage.metrics import structural_similarity as ssim
fm tensorflow.keras.applications import VGG19
fm tensorflow.keras.models import load_model, Model

# Judul dan sidebar
st.set_page_config(page_title="MLP-CNN Image Reconstruction Dashboard", layout="wide")
st.title("🔍 MLP-CNN Image Reconstruction Dashboard")
st.markdown("""
    <style>
    /* Ubah backgund keseluruhan */
    .stApp {
        backgund-color: #f5f7fb;
            font-family: 'Segoe UI', sans-serif;
    }

    /* Ubah warna judul */
    h1, h2, h3 {
        color: #1f3b75;
    }

    /* Styling untuk kotak metric */
    div[data-testid="metric-container"] {
        backgund-color: #ffffff;
        border: 1px solid #dfe4ea;
        padding: 15px;
        border-radius: 12px;
        box-shadow: 1px 1px 5px rgba(0,0,0,0.03);
        margin-bottom: 10px;
    }

    /* Warna teks label metric */
    div[data-testid="metric-container"] label {
        color: #1f3b75;
        font-weight: 600;
        font-size: 13px;
    }
    button {
        border-radius: 6px !important
    }
    </style>
""", unsafe_allow_html=True)

# Timeout durasi (dalam detik)
TIMEOUT_SECONDS = 600  # misalnya auto-mati dalam 60 detik jika tidak ada aktivitas

# Timestamp global yang akan diperbarui setiap rerun
last_active = time.time()

# Fungsi watchdog yang akan menghentikan pses jika timeout
def timeout_watcher():
    global last_active
    while True:
        time.sleep(10)  # Cek setiap 10 detik
        if time.time() - last_active > TIMEOUT_SECONDS:
            print("❌ Tidak ada aktivitas Streamlit. Menutup aplikasi...")
            os._exit(0)  # Paksa hentikan pses Python

# Jalankan watcher sekali saja saat script pertama kali dijalankan
if "watchdog_started" not in st.session_state:
    threading.Thread(target=timeout_watcher, daemon=True).start()
    st.session_state.watchdog_started = True

# Update waktu aktif setiap ada interaksi
last_active = time.time()

# === Berikut ini isi dashboard Streamlit ===
st.write("Dashboard ini akan otomatis tertutup jika tidak ada aktivitas selama 10 menit.")
st.button("Klik Aku Untuk Reset Timer")

# Navigasi
st.sidebar.header("Navigasi Dashboard")
section = st.sidebar.radio("Pilih Tampilan", [
    "📁 Dataset & Informasi",
    "🖼️ Rekonstruksi Citra",
    "📊 Evaluasi Kualitas",
    "🧾 Ringkasan Model"
])

if st.sidebar.button("🔄 Reset Cache Dataset"):
    st.session_state.reset_dataset = True
    st.cache_data.clear()
    st.rerun()

# ==============================
# CACHE DATASET
# ==============================
@st.cache_data()
def load_dataset():
    cache_file = "dataset_cache.npz"

    if "reset_dataset" in st.session_state and st.session_state.reset_dataset:
        if os.path.exists(cache_file):
            os.remove(cache_file)
        st.session_state.reset_dataset = False

    if os.path.exists(cache_file):
        data = np.load(cache_file)
        return data["G1"], data["images"], data["images2"]
    
    folder1_path = "C:/Users/TALITA/SSSKRIPPPSI/dataset/anom"
    folder2_path = "C:/Users/TALITA/SSSKRIPPPSI/dataset/gambar_ori"
    foldergmb2_path = "C:/Users/TALITA/SSSKRIPPPSI/gambar_rekonstruksi/gambar_rekonstruksi"
    
    # Load file Excel (anomali)
    G1_list = []
    for file_name in os.listdir(folder1_path):
        if file_name.endswith(".xlsx"):
            file_path = os.path.join(folder1_path, file_name)
            data = pd.read_excel(file_path, header=None).to_numpy().flatten()
            G1_list.append(data.reshape(-1, 1))
    G1 = np.array(G1_list).squeeze()
    # scaler = MinMaxScaler()
    # G1 = scaler.fit_transform(G1a)

    # Load gambar (Objek Asli)
    image_list = []
    for filename in os.listdir(folder2_path):
        if filename.lower().endswith((".png", ".jpg", ".jpeg")):
            img_path = os.path.join(folder2_path, filename)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (100, 100))
            img = img / 255.0
            image_list.append(img)
    image_array = np.array(image_list)

    # Load gambar (Linear Satu Langkah)
    image2_list = []
    for filename in sorted(os.listdir(foldergmb2_path)):
        if filename.endswith((".png", ".jpg", ".jpeg")):
            img_path = os.path.join(foldergmb2_path, filename)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (100, 100))
            img = img / 255.0
            if img is not None:
                image2_list.append(img)
    image2_array = np.array(image2_list)

    np.savez(cache_file, G1=G1, images=image_array, images2=image2_array)
    return G1, image_array, image2_array

G1, image_array, image2_array = load_dataset()
X_train1, X_test1, Y_train1, Y_test1 = train_test_split(G1, image_array, test_size=0.2, random_state=42)
X_train2, X_test2, Y_train2, Y_test2 = train_test_split(image_array, image2_array, test_size=0.2, random_state=42)

# =============================
# MODEL MLP-CNN
# =============================
if "model" not in st.session_state:
    vgg = VGG19(weights="imagenet", include_top=False)
    vgg.trainable = False
    loss_model = Model(inputs=vgg.input, outputs=vgg.get_layer("block3_conv3").output)

    def perceptual_loss(y_true, y_pred):
        return tf.reduce_mean(tf.abs(loss_model(y_true) - loss_model(y_pred)))
    def ssim_loss(y_true, y_pred):
        return 1 - tf.image.ssim(y_true, y_pred, max_val=1.0)
    def combined_loss(y_true, y_pred):
        return 0.4 * tf.keras.losses.MeanSquaredErr()(y_true, y_pred) + 0.3 * ssim_loss(y_true, y_pred) +  0.3 * perceptual_loss(y_true, y_pred)

    st.session_state.model = load_model(
        "C:/Users/TALITA/SSSKRIPPPSI/mlp_cnn_model100.keras",
        custom_objects={
            'combined_loss': combined_loss,
            'perceptual_loss': perceptual_loss,
            'ssim_loss': ssim_loss
        }
    )
model = st.session_state.model

# =============================
# CACHE PREDIKSI
# =============================
@st.cache_data()
def get_predictions():
    return model.predict(X_test1)
Y_pred_sample = get_predictions()

# =============================
# CACHE HISTORY
# =============================
@st.cache_data()
def load_history():
    with open("C:/Users/TALITA/SSSKRIPPPSI/history.pkl", "rb") as f:
        return pickle.load(f)

# =============================
# CACHE METRIK SSIM/PSNR
# =============================
@st.cache_data()
def compute_metrics(Y_true, Y_pred):
    ssim_scores, psnr_scores = [], []
    for y_true, y_pred in zip(Y_true, Y_pred):
        if y_pred.shape[-1] == 1:
            y_pred = np.repeat(y_pred, 3, axis=-1)
        if y_true.shape[-1] == 1:
            y_true = np.repeat(y_true, 3, axis=-1)
        ssim_score = ssim(y_true, y_pred, channel_axis=-1, data_range=1.0)
        psnr_score = psnr(y_true, y_pred, data_range=1.0)
        ssim_scores.append(ssim_score)
        psnr_scores.append(psnr_score)
    return ssim_scores, psnr_scores

# =============================
# 1. Dataset & Informasi
# =============================
if section == "📁 Dataset & Informasi":
    st.subheader("📁 Informasi Dataset")

    with st.spinner("📂 Memuat dataset..."):
        G1, image_array, image2_array = load_dataset()

    st.markdown(f"""
    - Jumlah objek numerik: **{len(G1)}**
    - Jumlah objek asli: **{len(image_array)}**
    - Ukuran citra: **{image_array[0].shape if len(image_array) > 0 else '(kosong)'}**
    - Dimensi fitur input: **{G1.shape}**
    """)

    # Contoh data
    st.write("📊 Contoh Data Objek Numerik (fitur):")
    st.dataframe(pd.DataFrame(G1[:7]).style.format("{:.6f}"))

    st.write("🖼️ Contoh Data Objek Asli:")
    st.image(image_array[:7], width=100, caption=[f"Sampel {i}" for i in range(7)])

# =============================
# 2. Rekonstruksi Citra
# =============================
elif section == "🖼️ Rekonstruksi Citra":
    st.subheader("🖼️ Perbandingan Citra Asli vs Rekonstruksi")
    st.markdown("Contoh hasil prediksi dari model:")
    num_samples = len(X_test1)
    idx = st.slider("🔢 Pilih Index Sampel Uji", 0, num_samples - 1, 0)

    # Ambil data satu sampel
    X_sample = X_test1[idx:idx+1]
    Y_true = Y_test1[idx]
    Y_pred = model.predict(X_sample)[0]
    Y_pred = np.clip(Y_pred, 0, 1)

    # Hitung metrik evaluasi
    # Validasi shape dan channel
    if Y_true.ndim == 2:
        Y_true = np.stack([Y_true]*3, axis=-1)
    if Y_pred.ndim == 2:
        Y_pred = np.stack([Y_pred]*3, axis=-1)
    elif Y_pred.shape[-1] == 1:
        Y_pred = np.repeat(Y_pred, 3, axis=-1)

    # Pastikan nilai dalam rentang 0-1
    Y_true = np.clip(Y_true, 0, 1)
    Y_pred = np.clip(Y_pred, 0, 1)

    # Hitung metrik
    ssim_score = ssim(Y_true, Y_pred, channel_axis=2, data_range=1.0)
    psnr_score = psnr(Y_true, Y_pred, data_range=1.0)
    mse_score = np.mean((Y_true - Y_pred) ** 2)

    # Tampilkan metrik
    col_m1, col_m2, col_m3 = st.columns(3)
    col_m1.metric("MSE", f"{mse_score:.4f}")
    col_m2.metric("🔍 SSIM", f"{ssim_score:.4f}")
    col_m3.metric("📏 PSNR", f"{psnr_score:.2f} dB")

    # Tampilkan gambar asli dan hasil prediksi
    col1, col2 = st.columns(2)
    with col1:
        st.image(Y_true, caption="🎯 Target (Asli)", width=200)
    with col2:
        st.image(Y_pred, caption="🧠 Prediksi (Model)", width=200)
    
    st.subheader("🔍 Perbandingan Citra Asli vs Rekonstruksi Linear Satu Langkah vs Rekonstruksi MLP-CNN")
    # Slider untuk pilih indeks gambar
    img_idx = st.slider("Pilih indeks gambar", 0, num_samples - 1, 0)

    # Ambil 1 gambar dari setiap jenis
    with st.spinner("🔍 Menghasilkan prediksi semua citra dengan MLP-CNN..."):
        Y_pred_all = model.predict(X_test1)
        Y_pred_all = np.clip(Y_pred_all, 0, 1)
    img_asli = Y_test1[img_idx]
    img_linear = Y_test2[img_idx]
    img_cnn = Y_pred_all[img_idx]

    # Konversi ke RGB kalau channel-nya cuma 1
    if img_asli.shape[-1] == 1:
        img_asli = np.repeat(img_asli, 3, axis=-1)
    if img_cnn.shape[-1] == 1:
        img_cnn = np.repeat(img_cnn, 3, axis=-1)
    if img_linear.shape[-1] == 1:
        img_linear = np.repeat(img_linear, 3, axis=-1)

    # Hitung metrik untuk linear
    mse_linear = np.mean((img_asli - img_linear) ** 2)
    ssim_linear = ssim(img_asli, img_linear, channel_axis=2, data_range=1.0)
    psnr_linear = psnr(img_asli, img_linear, data_range=1.0)

    # Hitung metrik untuk MLP-CNN
    mse_cnn = np.mean((img_asli - img_cnn) ** 2)
    ssim_cnn = ssim(img_asli, img_cnn, channel_axis=2, data_range=1.0)
    psnr_cnn = psnr(img_asli, img_cnn, data_range=1.0)

    # Tampilkan 3 gambar sejajar
    col1, col2, col3 = st.columns(3)
    with col1:
        st.image(img_asli, caption="Citra Asli", width=180)
        st.write(" ")
    with col2:
        st.image(img_linear, caption="Linear Satu Langkah", width=180)
        st.markdown(f"MSE: `{mse_linear:.4f}`  \nSSIM: `{ssim_linear:.4f}`  \nPSNR: `{psnr_linear:.2f} dB`")
    with col3:
        st.image(img_cnn, caption="MLP-CNN", width=180)
        st.markdown(f"MSE: `{mse_cnn:.4f}`  \nSSIM: `{ssim_cnn:.4f}`  \nPSNR: `{psnr_cnn:.2f} dB`")

# =============================
# 3. Evaluasi Kualitas
# =============================
elif section == "📊 Evaluasi Kualitas":
    st.subheader("📊 Evaluasi Rekonstruksi")

    st.markdown("""
    **Loss dan MSE (Mean Squared Err)** menunjukkan performa model selama pelatihan. Kurva menurun dan stabil mencerminkan pembelajaran yang baik.
                
    **SSIM (Structural Similarity Index)** mengukur kemiripan struktural antara citra asli dan hasil rekonstruksi — semakin mendekati 1, semakin mirip.

    **PSNR (Peak Signal-to-Noise Ratio)** mengukur kualitas rekonstruksi terhadap citra asli — semakin tinggi nilai dB, semakin baik kualitasnya.
    """)

    history = load_history()
    loss = history['loss']
    val_loss = history['val_loss']
    mse = history['mse']
    val_mse = history['val_mse']

    # Hitung metrik prediksi
    Y_pred_sample = get_predictions()
    Y_pred_sample = np.clip(Y_pred_sample, 0, 1)

    ssim_scores = []
    psnr_scores = []

    for y_true, y_pred in zip(Y_test1, Y_pred_sample):
        if y_pred.shape[-1] == 1:
            y_pred = np.repeat(y_pred, 3, axis=-1)
        if y_true.shape[-1] == 1:
            y_true = np.repeat(y_true, 3, axis=-1)

        ssim_score = ssim(y_true, y_pred, channel_axis=-1, data_range=1.0, win_size=11)
        psnr_score = psnr(y_true, y_pred, data_range=1.0)

        ssim_scores.append(ssim_score)
        psnr_scores.append(psnr_score)

    # ============================
    # Visualisasi dalam 1 kolom
    # ============================
    tab1, tab2, tab3, tab4 = st.tabs(["📉 Loss", "📉 MSE", "📊 SSIM", "📊 PSNR"])
    with tab1:
        st.markdown("**📉 Loss**")
        fig1, ax1 = plt.subplots()
        ax1.plot(loss, label='Training Loss', color='blue')
        ax1.plot(val_loss, label='Validation Loss', color='orange')
        ax1.set_title('Loss vs Epoch')
        ax1.set_xlabel('Epoch')
        ax1.set_ylabel('Loss')
        ax1.legend()
        ax1.grid(True)
        st.pyplot(fig1)
    with tab2:
        st.markdown("**📉 MSE**")
        fig2, ax2 = plt.subplots()
        ax2.plot(mse, label='Training MSE', color='green')
        ax2.plot(val_mse, label='Validation MSE', color='red')
        ax2.set_title('MSE vs Epoch')
        ax2.set_xlabel('Epoch')
        ax2.set_ylabel('Mean Squared Err')
        ax2.legend()
        ax2.grid(True)
        st.pyplot(fig2)
    with tab3:
        st.metric("Rata-rata SSIM", f"{np.mean(ssim_scores):.4f}")
        st.markdown("**📊 Histogram SSIM**")
        fig3, ax3 = plt.subplots()
        ax3.hist(ssim_scores, bins=20, color='skyblue', edgecolor='black')
        ax3.set_title("Distribusi SSIM")
        ax3.set_xlabel("SSIM")
        ax3.set_ylabel("Frekuensi")
        st.pyplot(fig3)
    with tab4:
        st.metric("Rata-rata PSNR", f"{np.mean(psnr_scores):.2f} dB")
        st.markdown("**📊 Histogram PSNR**")
        fig4, ax4 = plt.subplots()
        ax4.hist(psnr_scores, bins=20, color='salmon', edgecolor='black')
        ax4.set_title("Distribusi PSNR")
        ax4.set_xlabel("PSNR (dB)")
        ax4.set_ylabel("Frekuensi")
        st.pyplot(fig4)

# =============================
# 4. Ringkasan Model
# =============================
elif section == "🧾 Ringkasan Model":
    st.subheader("🧾 Arsitektur & Konfigurasi Model")
    st.markdown("""
    - **Encoder**: MLP
    - **Decoder**: CNN
    - **Loss Function**:
        - 40% MSE
        - 30% SSIM Loss
        - 30% Perceptual Loss (VGG19)
    """)

    image = Image.open("C:/Users/TALITA/Downloads/Cobalagi2.drawio.png")
    st.image(image, caption="Diagram Arsitektur Model MLP-CNN", use_container_width=True)
    # st.text(model.summary()) atau model.summary(print_fn=...)